In [ ]:
# y --> 0/1
# X & y
# Multi collinearity
#  independent variables are linearly relaed to log odds

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
sns.set(style='white')

In [ ]:
data = pd.read_csv('banking.csv')
data.head()

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data['loan'].value_counts()

In [ ]:
data['y'].value_counts()

In [ ]:
data.groupby('job').mean()

In [ ]:
data.groupby('y').mean()

In [ ]:
%matplotlib inline
pd.crosstab(data['job'],data['y']).plot(kind='bar')
plt.title("Purchase Freq of job title")
plt.xlabel("job")
plt.ylabel('Freq of purchase')

In [ ]:
%matplotlib inline
table = pd.crosstab(data['marital'],data['y'])
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title("marital")
plt.xlabel("marital")
plt.ylabel('Freq of purchase')

In [ ]:
%matplotlib inline
table = pd.crosstab(data['education'],data['y'])
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)

In [ ]:
data.head()

In [ ]:
data['age'].hist()

In [ ]:
data_final = pd.get_dummies(data, drop_first=True)
data_final.head()

In [ ]:
# Smote
X = data_final.drop(columns='y')
y = data_final['y']

In [ ]:
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
os_data_y['y'].value_counts(normalize=True)

In [ ]:
# Recursive Feature Elimination technique
from sklearn.feature_selection import RFE

logreg = LogisticRegression()
rfe = RFE(logreg, n_features_to_select=20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
cols = X.columns[rfe.support_]
cols

In [ ]:
X=os_data_X[cols]
y=os_data_y['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
# Evaluation metrics of Classification task

In [ ]:
# accuracy --> number of correct predictions/total number of predictions

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, f1_score, precision_score,recall_score

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm_display = ConfusionMatrixDisplay(cm, display_labels=[0,1])
cm_display.plot()
plt.show()

- True Negative (Top-Left Quadrant)
- False Positive (Top-Right Quadrant)
- False Negative (Bottom-Left Quadrant)
- True Positive (Bottom-Right Quadrant)

In [ ]:
print(f"Precision score is {precision_score(y_test,y_pred)}")
print(f"Recall score is {recall_score(y_test,y_pred)}")
print(f"f1-score is {f1_score(y_test,y_pred)}")

In [ ]:
# ROC AUC Score

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred_proba = logreg.predict_proba(X_test)
y_pred_proba

In [ ]:
roc_auc_score(y_test,y_pred_proba[:,1])

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr,tpr, threshoold = roc_curve(y_test,y_pred_proba[:,1])
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)
plt.plot(fpr, tpr, linestyle='--',color='orange', label='Logistic Regression')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show();